In [1]:
import polars as pl
import duckdb
import altair as alt
from pathlib import Path
import tomllib

In [2]:
with open("../src/monitor_campista/.streamlit/config.toml", "rb") as f:
    config = tomllib.load(f)

color_scale = config['theme']['colorScale']

In [3]:
con = duckdb.connect(Path('../data/03_gold/monitor_campista_pharma_ads_1880_1884.duckdb'), True)

In [4]:
list(con.sql("SHOW ALL TABLES;").pl()[:,2])

['alinhamento',
 'anuncios',
 'autorizacoes',
 'detalhamento_do_efeito',
 'detalhamento_forma_de_uso',
 'diagramacao',
 'discursos_de_autoridade',
 'doenca_mencionada',
 'duvidas',
 'elementos_de_composicao',
 'extras',
 'filhos',
 'hieraquia_da_informacao',
 'informacoes_indicativas',
 'mencoes_a_lugares',
 'origem',
 'original',
 'pai',
 'palavra_chave_efeito',
 'palavras_chave_produto',
 'primeiras_palavras_do_anuncio',
 'publico_mencionado',
 'responsavel_tecnico',
 'sinal_visual_de_autoridade',
 'substancias',
 'tipificacao_da_imagem_aprox',
 'tipo_de_produto',
 'variacao_tipografica',
 'variacao_typeface',
 'veiculacoes']

In [5]:
con.query("""
select 
    *
from veiculacoes
""").pl()

Identificador,Ano,Edição,Página,Coluna(s) ocupadas,Número de Colunas,Orientação,ano_edicao
str,i64,i64,i64,str,i64,str,str
"""tonico_oriental_cabello""",1880,2,4,"""1""",1,"""↥""","""1880_002"""
"""pilulas_vegeates_assucaradas_b…",1880,6,4,"""5""",1,"""↥""","""1880_006"""
"""novo_depurativo_extracto_salsa…",1880,6,3,"""4,5""",2,"""↥""","""1880_006"""
"""oleo_figado_bacalhau""",1880,8,4,"""5""",1,"""↥""","""1880_008"""
"""novo_depurativo_extracto_salsa…",1880,8,4,"""3,4""",2,"""↥""","""1880_008"""
…,…,…,…,…,…,…,…
"""cura_gonorrhea_sem_massantes_i…",1884,306,4,"""3""",1,"""↥""","""1884_306"""
"""oleo_ducoux_comprido""",1884,306,4,"""3""",1,"""↥""","""1884_306"""
"""importante_descoberta_xarope_s…",1884,306,4,"""3,4""",2,"""↥""","""1884_306"""


# Exploração geral

In [6]:
df_ads_by_edition = con.query("""
select 
    ano_edicao,
    count(*) as anuncios,
    min(Página) as menor_pagina,
    max(Página) as maior_pagina,
from veiculacoes
group by 
    ano_edicao
order by 
    ano_edicao
""").pl()

In [7]:
df_ads_by_edition

ano_edicao,anuncios,menor_pagina,maior_pagina
str,i64,i64,i64
"""1880_002""",1,4,4
"""1880_006""",2,3,4
"""1880_008""",2,4,4
"""1880_010""",1,4,4
"""1880_012""",2,4,4
…,…,…,…
"""1884_298""",7,3,4
"""1884_300""",9,3,4
"""1884_302""",8,3,4


In [8]:
df_ads_by_page = con.sql("""
    select
        "Página",
        count(*) as anuncios
    from veiculacoes
    group by
        "Página"
    order by
        Página
    """).pl()

In [9]:
# range_chart = (
#     alt.Chart(df_ads_by_edition)
#     .mark_area()
#     .encode(
#         x=alt.X("ano_edicao:O", title="Edition Year"),
#         y=alt.Y("pagina_primeiro_anuncio:Q", title="Page Number"),
#         y1="pagina_ultimo_anuncio:Q",
#         color=alt.Color("ano:N").scale(range=color_scale),
#     )
# )

# Add line for the first ad page
line_first = (
    alt.Chart(df_ads_by_edition)
    .mark_circle(color="darkblue")
    .encode(x="ano_edicao:O", y="pagina_primeiro_anuncio:Q")
)

# Add line for the last ad page
line_last = (
    alt.Chart(df_ads_by_edition)
    .mark_circle(color=color_scale[0])
    .encode(x="ano_edicao:O", y="pagina_ultimo_anuncio:Q")
)

# Combine all
chart = line_first + line_last

In [10]:
(
    alt.Chart(df_ads_by_page)
    .mark_bar(size=50)
    .encode(
        x=alt.X("Página:O")
        .title("Edição")
        .axis(labelAngle=0)
        .scale(domain=range(1, 9)),
        y=alt.Y("anuncios").title("Contagem Anúncios"),
    )
)

alt.Chart(...)

In [11]:
ad_edition_page = con.query("""
select 
    Ano,ano_edicao,Página
from veiculacoes
""").pl()

In [12]:
# something wrong with this graph
agg = ad_edition_page.group_by(["Página","Ano"]).len()
alt.Chart(agg).mark_bar().encode(
    x=alt.X('Página:O', title='Página'),
    y=alt.Y('count():Q', stack='zero', title='Páginas'),
    color='Ano:N'
)

alt.Chart(...)

In [14]:
(alt.Chart(ad_edition_page).mark_bar()
.transform_aggregate(
    count='count()',
    groupby=['Ano', 'Página']
).encode(
    x='Página:N',
    y=alt.Y('sum(count):Q', title="Veiculações"),
    color=alt.Color('Ano:N').scale(range=color_scale[1::1])
))

alt.Chart(...)

In [15]:
ads_per_edition = (
    alt.Chart(df_ads_by_page)
    .mark_bar(size=70)
    .encode(
        x=alt.X("Página:O")
        .title("Edição")
        .axis(labelAngle=0)
        .scale(domain=range(1, 9)),
        y=alt.Y("anuncios")
        .title("Contagem Anúncios")
        .axis(format=",.0f"),  # Basic formatting without thousands separator
        color=alt.value(color_scale[0]),  # Apply a single color to all bars
    )
)
ads_per_edition

alt.Chart(...)

# Doenças

In [65]:
df_ailments_per_ad = con.sql("""
    select
        doenca_mencionada as Doença,
        count(distinct Identificador) as Anúncios,
        count(distinct ano_edicao) as Veiculações
    from 
        doenca_mencionada
    left join
        veiculacoes using(Identificador)
    group by 
        doenca_mencionada
    order by
        Anúncios desc
""").pl()
df_ailments_per_ad

Doença,Anúncios,Veiculações
str,i64,i64
"""Rheumatismo""",20,312
"""Ausente""",20,160
"""Anemia""",14,232
"""Escrofula (escrophulasa scroph…",13,233
"""Chlorose [chlorosis]""",13,215
…,…,…
"""Consequencias do parto""",1,6
"""Pallidez""",1,36
"""Ronquidões""",1,10


In [67]:
(
    alt.Chart(df_ailments_per_ad)
        .mark_bar()
        .encode(
        x=alt.X("Doença", sort="-y"),
        y=alt.Y("Anúncios"),
        color=alt.Color("Veiculações")
    )
)

alt.Chart(...)

In [95]:
long = (
    df_ailments_per_ad
    .unpivot(
        index='Doença',                # columns to keep as-is
        on=['Anúncios', 'Veiculações'], # columns to stack
        variable_name='Métrica',        # name for the “former column name” column
        value_name='Valor'              # name for the stacked values
    )
)

alt.Chart(long).mark_bar().encode(
    x=alt.X('Doença:N', sort='-y'),
    y='Valor:Q',
    color='Métrica:N'
)

alt.Chart(...)

In [108]:
alt.Chart(df_ailments_per_ad).mark_circle(size=200).encode(
    x=alt.X('Anúncios:Q', title='Anúncios'),
    y=alt.Y('Veiculações:Q', title='Veiculações'),
    tooltip=['Doença','Anúncios','Veiculações'],
    color=alt.Color(
        'Doença',
        scale=alt.Scale(range=color_scale)   
    ),
)

alt.Chart(...)

In [101]:
base = alt.Chart(df_ailments_per_ad).encode(
    x=alt.X('Anúncios:Q', title='Anúncios'),
    y=alt.Y('Veiculações:Q', title='Veiculações')
)

points = base.mark_circle(size=200)
labels = base.mark_text(
    align='center',
    baseline='bottom',
    dy=-8,               # nudge text up a little
    fontSize=9
).encode(text='Doença:N')

points + labels

alt.LayerChart(...)

In [79]:
# 1. Create a base chart that defines the shared X-axis
base = alt.Chart(df_ailments_per_ad).encode(
    x=alt.X("Doença:N", sort="-y")  # 'N' for Nominal (discrete) data
)

# 2. Create the first chart for "Anúncios" (e.g., as bars)
bars = base.mark_bar(color=color_scale[1]).encode(
    y=alt.Y("Anúncios:Q").axis(titleColor=color_scale[0])  # 'Q' for Quantitative data
)

# 3. Create the second chart for "Veiculações" (e.g., as a line)
line = base.mark_bar(color=color_scale[3]).encode(
    y=alt.Y("Veiculações:Q").axis(titleColor=color_scale[1])
)

# 4. Layer them and create independent y-axes
final_chart = alt.layer(bars, line).resolve_scale(y="independent")

final_chart

alt.LayerChart(...)

NameError: name 'df_long' is not defined

In [34]:
df_ailments_count_per_ad = con.sql("""
    select
        Identificador as anuncio,
        count(distinct doenca_mencionada) as doencas,
    from 
        doenca_mencionada
    group by 
        Identificador,
""").pl()
df_ailments_count_per_ad

anuncio,doencas
str,i64
"""xarope_massa""",8
"""escrophulas""",1
"""tosses_bronchites""",7
"""essencia_depurativa_ferruminos…",1
"""exposicao_paris_cura_asma""",1
…,…
"""pomada_vaseline""",1
"""chlorose_anemias_cores_FERRO_B…",4
"""oleo_figado_bacalhau""",1


In [37]:
(
    alt.Chart(df_ailments_count_per_ad)
    .mark_bar()
    .encode(
        x=alt.X('doencas:N', title='Contagem doenças mencionadas', sort='x')
        .axis(labelAngle=0),
        y=alt.Y('count()', title='Anúncios'),
        color=alt.Color(value=color_scale[0])
    )
)

alt.Chart(...)

In [36]:
(
    alt.Chart(df_ailments_count_per_ad)
    .mark_bar()
    .encode(
        x=alt.X('doencas:N', title='Contagem doenças mencionadas', sort='x')
            .axis(labelAngle=0),
        y=alt.Y('count()', title='Anúncios'),
        # color=alt.Color("Ano:N").scale(range=["grey"] + color_scale)
    )
         .properties(title="Doenças por Anúncio")
)

alt.Chart(...)

In [ ]:
# con.close()